In [3]:
## DATA INGESTION
from langchain_community.document_loaders import TextLoader
loader = TextLoader('speech.txt')
doc = loader.load()  # converts it into a text document object
doc[0].page_content


'\nHi, My name is Ayush Raj.'

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')


In [5]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
  
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Dog")
docs = loader.load()
docs[0].page_content

'\n\n\n\nDog - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nTaxonomy\n\n\n\n\nToggle Taxonomy subsection\n\n\n\n\n\n1.1\nDomestication\n\n\n\n\n\n\n\n\n1.2\nBreeds\n\n\n\n\n\n\n\n\n\n\n2\nAnatomy and physiology\n

In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("att1.pdf")
docs = loader.load()
docs

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'att1.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser

In [62]:
# EMBED

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(docs)

In [72]:
## VECTOR EMBEDDINGS and VECTOR STORE

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
# shutil.rmtree("./chroma_db", ignore_errors=True)
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")
# db = Chroma.from_documents(documents, embeddings )
db = FAISS.from_documents(documents[:30], embeddings )
db


In [14]:
query = "An attention function can be described as mapping a query"
results = db.similarity_search(query)
results[0].page_content


'Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the\nquery with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the\nquery with all keys, divide each by √dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together\ninto a matrix Q. The keys and values are also packed together into matrices K and V . We compute\nthe matrix of outputs as:\nAttention(Q, K, V) = softmax(QKT\n√dk\n)V (1)'

In [10]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
llm

Ollama()

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
     Answer the following question based only on the context provided.
     Think step by step before answering. I will tip you $1000 if the user finds it helpful.
     <context>
     {context}
     </context>
     Question: {input}""")


In [7]:
# chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt=prompt)

In [8]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10e547d50>, search_kwargs={})

In [9]:
# retrieval chain
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [1]:
retrieval_chain.invoke({"input": "Explain attention mechanism in simple terms?"})

NameError: name 'retrieval_chain' is not defined

In [5]:
import google.generativeai as genai
import os

# Replace with your actual API key or retrieve from environment variable
genai.configure(api_key=os.getenv('GOOGLE_API_KEY')) 

model = genai.GenerativeModel("gemini-2.0-flash")
prompt = "What is the capital of France?"
response = model.generate_content(prompt)
print(response.text)

The capital of France is Paris.

